In [ ]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.apps import apps as django_apps
from django.db.models.signals import post_save, post_delete
from edc_base.utils import get_utcnow
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT
from edc_visit_schedule import site_visit_schedules

from copy import copy

In [ ]:
subject_identifier = 'R142-040990113-8'
child_subject_identifier = 'B142-040990113-8-10'

In [ ]:
def update_forms_visit(forms, visit=None, new_visit=None, form_type='crf'):
    for form in forms:
        kwargs = {}
        model_cls = django_apps.get_model(form.model)
        if form_type == 'requisition':
            kwargs = {'panel__name': form.panel_name}
            post_save.disconnect(dispatch_uid='senaite_sample_create_on_post_save')
        try:
            model_obj = model_cls.objects.get(maternal_visit=visit, **kwargs)
        except model_cls.DoesNotExist:
            continue
        else:
            print(model_obj)
            model_obj.maternal_visit = new_visit
            model_obj.save()

In [ ]:
appt_list = ['appt_type', 'appt_status', 'appt_reason', 'comment', 'appt_datetime']

In [ ]:
visit = ChildVisit.objects.get(subject_identifier=child_subject_identifier, visit_code='2000')
enrol_dt = visit.report_datetime

In [ ]:
post_save.disconnect(dispatch_uid='maternal_visit_on_post_save')


onschedules = SubjectScheduleHistory.objects.filter(
    subject_identifier=subject_identifier).exclude(created__date=get_utcnow().date()).order_by('onschedule_datetime')
for onschedule in onschedules:
    print(onschedule.schedule_name)
    import pdb;pdb.set_trace()
    try:
        csh = CohortSchedules.objects.get(schedule_name=onschedule.schedule_name)
    except CohortSchedules.DoesNotExist:
        print('meh')
    else:
        swap_csh = CohortSchedules.objects.get(
            schedule_type=csh.schedule_type,
            child_count=csh.child_count,
            cohort_name='cohort_c')
        _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
            onschedule_model=swap_csh.onschedule_model, name=swap_csh.schedule_name)

        if not schedule.is_onschedule(
            subject_identifier=subject_identifier, report_datetime=onschedule.onschedule_datetime):
            base_appt_datetime = onschedule.onschedule_datetime
            if 'quart' in swap_csh.schedule_name:
                base_appt_datetime = enrol_dt
            schedule.put_on_schedule(
                subject_identifier=onschedule.subject_identifier, 
                onschedule_datetime=onschedule.onschedule_datetime,
                schedule_name=swap_csh.schedule_name,
                base_appt_datetime=base_appt_datetime)
            onschedule_cls = django_apps.get_model(swap_csh.onschedule_model)

            onsch = onschedule_cls.objects.get(
                subject_identifier=subject_identifier,
                schedule_name=swap_csh.schedule_name)
            onsch.child_subject_identifier=child_subject_identifier
            onsch.onschedule_datetime = onschedule.onschedule_datetime
            onsch.save()

            complete_appts = Appointment.objects.filter(
                subject_identifier=subject_identifier,
                schedule_name=onschedule.schedule_name, ).exclude(appt_status=NEW_APPT)

            for appt in complete_appts:
                # Get corresponding appt
                new_appt = Appointment.objects.filter(subject_identifier=subject_identifier,
                                                      visit_code=appt.visit_code,
                                                      visit_code_sequence=appt.visit_code_sequence,
                                                      schedule_name=swap_csh.schedule_name)
                if new_appt.exists():
                    appt_dict = {key: getattr(appt, key) for key in appt_list}
                    new_appt.update(**appt_dict)
                    try:
                        visit = MaternalVisit.objects.get(appointment=appt)
                    except MaternalVisit.DoesNotExist:
                        continue
                    else:
                        new_visit = copy(visit)
                        new_visit.pk = None
                        new_visit.appointment = new_appt[0]
                        new_visit.schedule_name = new_appt[0].schedule_name
                        new_visit.visit_schedule_name = new_appt[0].visit_schedule_name
                        new_visit.save()

                        crfs = CrfMetadata.objects.filter(
                            subject_identifier=subject_identifier,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        print(visit, new_visit)
                        update_forms_visit(forms=crfs, visit=visit, new_visit=new_visit)

                        reqs = RequisitionMetadata.objects.filter(
                            subject_identifier=subject_identifier,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        update_forms_visit(forms=reqs, visit=visit, new_visit=new_visit, form_type='requisition')
                        visit.save()

In [ ]:
post_delete.disconnect(dispatch_uid='metadata_reset_on_post_delete')

onschedule_cls = [OnScheduleCohortBSec, OnScheduleCohortBSecQuart]
for onsch_cls in onschedule_cls:
    onsch = onsch_cls.objects.get(subject_identifier=subject_identifier)
    ssh = SubjectScheduleHistory.objects.get(subject_identifier=subject_identifier,
                                             schedule_name=onsch.schedule_name)
    appts = Appointment.objects.filter(subject_identifier=subject_identifier,
                                       schedule_name=onsch.schedule_name)
    for appt in appts:
        MaternalVisit.objects.filter(appointment=appt).delete()
    appts.delete()
    ssh.delete()
    onsch.delete()

In [ ]:
visit_code = 2001
while visit_code < 2014:
    appt = Appointment.objects.get(
        subject_identifier=subject_identifier, visit_code=f'{visit_code}M')
    appt.delete()
    visit_code += 1